In [1]:
import os
import gc
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import utils
import scoring
import catboost
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold

EMPTY_FILLER = 10**9

In [2]:
full_train = pd.read_csv("../data/train_v2.csv", usecols=(77,79))

In [3]:
full_train.shape

(5445705, 2)

In [4]:
wieght_filter = full_train['weight'] >= 0

In [5]:
label, weight = full_train['label'], full_train['weight']
del full_train

In [7]:
SIMPLE_PLUS_FOI_FEAT_COLS = (['{}[{}]'.format(name, station) 
                                        for name in (
                                            ['ncl', 'avg_cs', 'dist_FOI_hit_XY', 'dist_FOI_hit_XYZ'] + #'dist_FOI_hit_T',
                                            ['MatchedHit_%s' % typ for typ in 
                                                ['TYPE', 'DX', 'DY', 'DZ',  'DT'] #'X', 'Y', 'Z','T',
                                            ] + 
                                            ['Mextra_D%s2' % typ for typ in ['X', 'Y']] + 
#                                             ['Lextra_%s' % typ for typ in ['X', 'Y']] + 
                                            ['closest_%s' % typ for typ in ['x', 'y', 'z', 'T', 'dx', 'dy']] +
                                            ['closest_dist_%s' % typ for typ in ['x', 'y']]
                                        ) for station in range(4)
                                ] + 
                                ['NShared',
                                'square_deviation', 'square_deviation_FOI', 
                                'diff_of_square_deviations',
                                'FOI_hits_N', 'PT', 'P'] + 
                                [ 'deviation', 'length_MHit', 
                                  'deviation_FOI', 'length_FOI',
                                  'diff_of_deviations' ,'diff_of_lengths', #'isMuon'
                                ] + 
                                ['momentum_angle', 'FOI_vect_angle', 'mhit_vect_angle', 
                                'diff_FOI_angle', 'diff_mhit_angle', 
                                'time_estim', 'time_estim_closest'])

In [8]:
len(SIMPLE_PLUS_FOI_FEAT_COLS)

96

In [8]:
ship_indexes = weight[weight < 0].index + 1
ship_indexes.shape

(776386,)

In [9]:
full_train = pd.read_csv("../additional_feats_for_train_v2.csv", skiprows=ship_indexes)

In [10]:
full_train.shape

(4669319, 124)

In [12]:
abs_weights = np.abs(weight[wieght_filter].reset_index()['weight'])

In [14]:
abs_weights.shape

(4669319,)

In [15]:
label.shape

(5445705,)

In [16]:
skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)

In [17]:
i = 0
for train_index, test_index in skf.split(np.zeros(full_train.shape[0]), label[wieght_filter]):
    model = catboost.CatBoostClassifier(iterations=550, max_depth=7, border_count=254,task_type='GPU', verbose=True, random_seed=42*i)
    model.fit(full_train.iloc[train_index].loc[:,SIMPLE_PLUS_FOI_FEAT_COLS], 
              label[wieght_filter].reset_index()['label'].iloc[train_index], 
              sample_weight=abs_weights.iloc[train_index], 
             eval_set=(full_train.iloc[test_index].loc[:, SIMPLE_PLUS_FOI_FEAT_COLS], 
                       label[wieght_filter].reset_index()['label'].iloc[test_index]))
    print('score: %f' % 
          scoring.rejection90(label[wieght_filter].reset_index()['label'].iloc[test_index].values, 
                              model.predict_proba( full_train.iloc[test_index, :].loc[:,SIMPLE_PLUS_FOI_FEAT_COLS] )[:, 1],
                              abs_weights.iloc[test_index].values))
    model_name = "track_1_model_cat_7depth_550_iter_feats4_7fold%d" % i
    model.save_model("../models/{}.cbm".format(model_name))
    gc.collect()
    i+= 1

Learning rate set to 0.240779
0:	learn: 0.6210003	test: 0.6530846	best: 0.6530846 (0)	total: 439ms	remaining: 4m 1s
1:	learn: 0.5822561	test: 0.6273178	best: 0.6273178 (1)	total: 792ms	remaining: 3m 37s
2:	learn: 0.5554681	test: 0.6070005	best: 0.6070005 (2)	total: 1.2s	remaining: 3m 38s
3:	learn: 0.5394470	test: 0.5939018	best: 0.5939018 (3)	total: 1.52s	remaining: 3m 27s
4:	learn: 0.5233514	test: 0.5797265	best: 0.5797265 (4)	total: 1.91s	remaining: 3m 28s
5:	learn: 0.5117900	test: 0.5688334	best: 0.5688334 (5)	total: 2.34s	remaining: 3m 31s
6:	learn: 0.5023618	test: 0.5595442	best: 0.5595442 (6)	total: 2.68s	remaining: 3m 28s
7:	learn: 0.4961203	test: 0.5525030	best: 0.5525030 (7)	total: 3.04s	remaining: 3m 26s
8:	learn: 0.4906691	test: 0.5462654	best: 0.5462654 (8)	total: 3.41s	remaining: 3m 25s
9:	learn: 0.4855043	test: 0.5403094	best: 0.5403094 (9)	total: 3.76s	remaining: 3m 22s
10:	learn: 0.4809949	test: 0.5348702	best: 0.5348702 (10)	total: 4.1s	remaining: 3m 20s
11:	learn: 0.4

93:	learn: 0.4116781	test: 0.4519602	best: 0.4519602 (93)	total: 33.8s	remaining: 2m 44s
94:	learn: 0.4113676	test: 0.4516348	best: 0.4516348 (94)	total: 34.2s	remaining: 2m 43s
95:	learn: 0.4109714	test: 0.4512197	best: 0.4512197 (95)	total: 34.6s	remaining: 2m 43s
96:	learn: 0.4106967	test: 0.4509381	best: 0.4509381 (96)	total: 34.9s	remaining: 2m 43s
97:	learn: 0.4104756	test: 0.4507104	best: 0.4507104 (97)	total: 35.2s	remaining: 2m 42s
98:	learn: 0.4100371	test: 0.4502518	best: 0.4502518 (98)	total: 35.6s	remaining: 2m 42s
99:	learn: 0.4098303	test: 0.4500391	best: 0.4500391 (99)	total: 36s	remaining: 2m 41s
100:	learn: 0.4095793	test: 0.4498330	best: 0.4498330 (100)	total: 36.3s	remaining: 2m 41s
101:	learn: 0.4091480	test: 0.4493935	best: 0.4493935 (101)	total: 36.7s	remaining: 2m 41s
102:	learn: 0.4088580	test: 0.4491018	best: 0.4491018 (102)	total: 37.1s	remaining: 2m 41s
103:	learn: 0.4083464	test: 0.4485011	best: 0.4485011 (103)	total: 37.5s	remaining: 2m 40s
104:	learn: 0.4

184:	learn: 0.3869024	test: 0.4271527	best: 0.4271527 (184)	total: 1m 7s	remaining: 2m 13s
185:	learn: 0.3866355	test: 0.4268958	best: 0.4268958 (185)	total: 1m 7s	remaining: 2m 12s
186:	learn: 0.3863494	test: 0.4266371	best: 0.4266371 (186)	total: 1m 8s	remaining: 2m 12s
187:	learn: 0.3861237	test: 0.4263989	best: 0.4263989 (187)	total: 1m 8s	remaining: 2m 12s
188:	learn: 0.3858772	test: 0.4261637	best: 0.4261637 (188)	total: 1m 8s	remaining: 2m 11s
189:	learn: 0.3856613	test: 0.4259564	best: 0.4259564 (189)	total: 1m 9s	remaining: 2m 11s
190:	learn: 0.3853344	test: 0.4256111	best: 0.4256111 (190)	total: 1m 9s	remaining: 2m 11s
191:	learn: 0.3851286	test: 0.4253945	best: 0.4253945 (191)	total: 1m 10s	remaining: 2m 10s
192:	learn: 0.3848810	test: 0.4251706	best: 0.4251706 (192)	total: 1m 10s	remaining: 2m 10s
193:	learn: 0.3846114	test: 0.4249055	best: 0.4249055 (193)	total: 1m 10s	remaining: 2m 9s
194:	learn: 0.3844562	test: 0.4247959	best: 0.4247959 (194)	total: 1m 11s	remaining: 2m 

274:	learn: 0.3679799	test: 0.4084063	best: 0.4084063 (274)	total: 1m 40s	remaining: 1m 40s
275:	learn: 0.3678450	test: 0.4082477	best: 0.4082477 (275)	total: 1m 41s	remaining: 1m 40s
276:	learn: 0.3676691	test: 0.4080990	best: 0.4080990 (276)	total: 1m 41s	remaining: 1m 39s
277:	learn: 0.3674324	test: 0.4078607	best: 0.4078607 (277)	total: 1m 41s	remaining: 1m 39s
278:	learn: 0.3671683	test: 0.4076089	best: 0.4076089 (278)	total: 1m 42s	remaining: 1m 39s
279:	learn: 0.3669426	test: 0.4073942	best: 0.4073942 (279)	total: 1m 42s	remaining: 1m 38s
280:	learn: 0.3667212	test: 0.4071930	best: 0.4071930 (280)	total: 1m 43s	remaining: 1m 38s
281:	learn: 0.3664401	test: 0.4068691	best: 0.4068691 (281)	total: 1m 43s	remaining: 1m 38s
282:	learn: 0.3663229	test: 0.4067853	best: 0.4067853 (282)	total: 1m 43s	remaining: 1m 37s
283:	learn: 0.3660852	test: 0.4065555	best: 0.4065555 (283)	total: 1m 44s	remaining: 1m 37s
284:	learn: 0.3659445	test: 0.4064533	best: 0.4064533 (284)	total: 1m 44s	remain

364:	learn: 0.3518954	test: 0.3931791	best: 0.3931791 (364)	total: 2m 14s	remaining: 1m 8s
365:	learn: 0.3517264	test: 0.3930105	best: 0.3930105 (365)	total: 2m 15s	remaining: 1m 7s
366:	learn: 0.3515292	test: 0.3928304	best: 0.3928304 (366)	total: 2m 15s	remaining: 1m 7s
367:	learn: 0.3513716	test: 0.3926953	best: 0.3926953 (367)	total: 2m 15s	remaining: 1m 7s
368:	learn: 0.3511685	test: 0.3925028	best: 0.3925028 (368)	total: 2m 16s	remaining: 1m 6s
369:	learn: 0.3510138	test: 0.3923496	best: 0.3923496 (369)	total: 2m 16s	remaining: 1m 6s
370:	learn: 0.3508756	test: 0.3922306	best: 0.3922306 (370)	total: 2m 17s	remaining: 1m 6s
371:	learn: 0.3507292	test: 0.3921063	best: 0.3921063 (371)	total: 2m 17s	remaining: 1m 5s
372:	learn: 0.3505945	test: 0.3919574	best: 0.3919574 (372)	total: 2m 17s	remaining: 1m 5s
373:	learn: 0.3504758	test: 0.3918799	best: 0.3918799 (373)	total: 2m 18s	remaining: 1m 4s
374:	learn: 0.3503305	test: 0.3917562	best: 0.3917562 (374)	total: 2m 18s	remaining: 1m 4s

...

154:	learn: 0.3944067	test: 0.4326760	best: 0.4326760 (154)	total: 55.6s	remaining: 2m 21s
155:	learn: 0.3941846	test: 0.4324946	best: 0.4324946 (155)	total: 56s	remaining: 2m 21s
156:	learn: 0.3939052	test: 0.4322490	best: 0.4322490 (156)	total: 56.4s	remaining: 2m 21s
157:	learn: 0.3936096	test: 0.4319720	best: 0.4319720 (157)	total: 56.8s	remaining: 2m 20s
158:	learn: 0.3934086	test: 0.4317624	best: 0.4317624 (158)	total: 57.1s	remaining: 2m 20s
159:	learn: 0.3932042	test: 0.4315894	best: 0.4315894 (159)	total: 57.5s	remaining: 2m 20s
160:	learn: 0.3928666	test: 0.4312557	best: 0.4312557 (160)	total: 58s	remaining: 2m 20s
161:	learn: 0.3926752	test: 0.4310854	best: 0.4310854 (161)	total: 58.4s	remaining: 2m 19s
162:	learn: 0.3923993	test: 0.4308188	best: 0.4308188 (162)	total: 58.8s	remaining: 2m 19s
163:	learn: 0.3922257	test: 0.4306272	best: 0.4306272 (163)	total: 59.2s	remaining: 2m 19s
164:	learn: 0.3920288	test: 0.4304326	best: 0.4304326 (164)	total: 59.5s	remaining: 2m 18s
165

244:	learn: 0.3744003	test: 0.4132871	best: 0.4132871 (244)	total: 1m 29s	remaining: 1m 51s
245:	learn: 0.3741142	test: 0.4129882	best: 0.4129882 (245)	total: 1m 30s	remaining: 1m 51s
246:	learn: 0.3739063	test: 0.4128010	best: 0.4128010 (246)	total: 1m 30s	remaining: 1m 51s
247:	learn: 0.3737304	test: 0.4126438	best: 0.4126438 (247)	total: 1m 30s	remaining: 1m 50s
248:	learn: 0.3734707	test: 0.4123817	best: 0.4123817 (248)	total: 1m 31s	remaining: 1m 50s
249:	learn: 0.3732912	test: 0.4122248	best: 0.4122248 (249)	total: 1m 31s	remaining: 1m 50s
250:	learn: 0.3730719	test: 0.4120164	best: 0.4120164 (250)	total: 1m 32s	remaining: 1m 49s
251:	learn: 0.3728556	test: 0.4118367	best: 0.4118367 (251)	total: 1m 32s	remaining: 1m 49s
252:	learn: 0.3726917	test: 0.4116801	best: 0.4116801 (252)	total: 1m 32s	remaining: 1m 49s
253:	learn: 0.3725313	test: 0.4115447	best: 0.4115447 (253)	total: 1m 33s	remaining: 1m 48s
254:	learn: 0.3723235	test: 0.4113464	best: 0.4113464 (254)	total: 1m 33s	remain

334:	learn: 0.3580396	test: 0.3977238	best: 0.3977238 (334)	total: 2m 3s	remaining: 1m 19s
335:	learn: 0.3578767	test: 0.3975781	best: 0.3975781 (335)	total: 2m 4s	remaining: 1m 19s
336:	learn: 0.3577355	test: 0.3974526	best: 0.3974526 (336)	total: 2m 4s	remaining: 1m 18s
337:	learn: 0.3576002	test: 0.3973314	best: 0.3973314 (337)	total: 2m 5s	remaining: 1m 18s
338:	learn: 0.3574551	test: 0.3972159	best: 0.3972159 (338)	total: 2m 5s	remaining: 1m 18s
339:	learn: 0.3572827	test: 0.3970441	best: 0.3970441 (339)	total: 2m 5s	remaining: 1m 17s
340:	learn: 0.3570928	test: 0.3968724	best: 0.3968724 (340)	total: 2m 6s	remaining: 1m 17s
341:	learn: 0.3569302	test: 0.3967232	best: 0.3967232 (341)	total: 2m 6s	remaining: 1m 17s
342:	learn: 0.3567503	test: 0.3965508	best: 0.3965508 (342)	total: 2m 7s	remaining: 1m 16s
343:	learn: 0.3566124	test: 0.3964068	best: 0.3964068 (343)	total: 2m 7s	remaining: 1m 16s
344:	learn: 0.3564231	test: 0.3962044	best: 0.3962044 (344)	total: 2m 7s	remaining: 1m 15s

425:	learn: 0.3433964	test: 0.3836441	best: 0.3836441 (425)	total: 2m 39s	remaining: 46.4s
426:	learn: 0.3432536	test: 0.3835048	best: 0.3835048 (426)	total: 2m 39s	remaining: 46.1s
427:	learn: 0.3431138	test: 0.3833749	best: 0.3833749 (427)	total: 2m 40s	remaining: 45.7s
428:	learn: 0.3429612	test: 0.3832181	best: 0.3832181 (428)	total: 2m 40s	remaining: 45.4s
429:	learn: 0.3428378	test: 0.3831085	best: 0.3831085 (429)	total: 2m 41s	remaining: 45s
430:	learn: 0.3426933	test: 0.3829872	best: 0.3829872 (430)	total: 2m 41s	remaining: 44.6s
431:	learn: 0.3426111	test: 0.3829136	best: 0.3829136 (431)	total: 2m 41s	remaining: 44.2s
432:	learn: 0.3424586	test: 0.3827846	best: 0.3827846 (432)	total: 2m 42s	remaining: 43.9s
433:	learn: 0.3423320	test: 0.3826871	best: 0.3826871 (433)	total: 2m 42s	remaining: 43.5s
434:	learn: 0.3421227	test: 0.3824557	best: 0.3824557 (434)	total: 2m 43s	remaining: 43.1s
435:	learn: 0.3419849	test: 0.3823502	best: 0.3823502 (435)	total: 2m 43s	remaining: 42.8s
4

516:	learn: 0.3311283	test: 0.3720804	best: 0.3720804 (516)	total: 3m 16s	remaining: 12.5s
517:	learn: 0.3309564	test: 0.3719135	best: 0.3719135 (517)	total: 3m 16s	remaining: 12.1s
518:	learn: 0.3308481	test: 0.3718175	best: 0.3718175 (518)	total: 3m 16s	remaining: 11.8s
519:	learn: 0.3306612	test: 0.3716399	best: 0.3716399 (519)	total: 3m 17s	remaining: 11.4s
520:	learn: 0.3305118	test: 0.3715078	best: 0.3715078 (520)	total: 3m 17s	remaining: 11s
521:	learn: 0.3304031	test: 0.3714073	best: 0.3714073 (521)	total: 3m 18s	remaining: 10.6s
522:	learn: 0.3302367	test: 0.3712539	best: 0.3712539 (522)	total: 3m 18s	remaining: 10.3s
523:	learn: 0.3300610	test: 0.3710891	best: 0.3710891 (523)	total: 3m 19s	remaining: 9.89s
524:	learn: 0.3299217	test: 0.3709631	best: 0.3709631 (524)	total: 3m 19s	remaining: 9.51s
525:	learn: 0.3298170	test: 0.3708754	best: 0.3708754 (525)	total: 3m 20s	remaining: 9.13s
526:	learn: 0.3296698	test: 0.3707367	best: 0.3707367 (526)	total: 3m 20s	remaining: 8.75s
5